In [40]:
import pickle
import pandas as pd

from keras_preprocessing.text import Tokenizer

In [77]:
images_folder = 'D:/datasets/flickr-30k-images'

curr_folder = "D:/YandexDisk/datasets/"
path_train = curr_folder + "captions-ru-train.csv"

path_features = curr_folder + "features.pkl"
path_sentences = curr_folder + "sentences-train.pkl"

# Загрузка данных

Мы собираемся обучить данные по всем фотографиям и подписям в обучающем наборе данных. Во время обучения мы будем отслеживать производительность модели в наборе данных разработки и использовать эту производительность, чтобы решить, когда сохранять модели в файл.

Модель, которую мы разработаем, будет генерировать подпись к фотографии, и подпись будет генерироваться по одному слову за раз.

Последовательность ранее сгенерированных слов будет предоставлена в качестве входных данных. Поэтому нам понадобится "первое слово", чтобы начать процесс генерации, и "последнее слово", чтобы сигнализировать об окончании подписи. Для этой цели мы будем использовать строки "startseq" и "endseq". Эти маркеры добавляются к загруженным описаниям по мере их загрузки. Важно сделать это сейчас, прежде чем мы закодируем текст, чтобы токены также были закодированы правильно.

# Множество изображений

In [65]:
def image_names_set(data):
    vals = set()

    for idx in data.index:
        vals.add(data.iat[idx, 0][:-4])

    return vals

def load_image_features(filename, data):
    all_features = pickle.load(open(filename, 'rb'))
    features = {k: all_features[k] for k in data}

    return features

In [79]:
train_df = pd.read_csv(path_train, delimiter='|')
train_features = load_image_features(path_features, image_names_set(train_df))
print('кол-во изображений ... %d' % len(train_features))

кол-во изображений ... 10902


# Закодировать знаки в числа

Текст описания необходимо будет закодировать в числа, прежде чем его можно будет представить модели в качестве входных данных или сравнить с предсказаниями модели.

Первым шагом в кодировании данных является создание согласованного сопоставления слов с уникальными целочисленными значениями. Keras предоставляет класс Tokenizer, который может изучить это сопоставление из загруженных данных описания.

In [18]:
with open (path_sentences, 'rb') as f:
    sentences = pickle.load(f)

In [23]:
def create_tokenizer(a_sentences):
    tokenizer = Tokenizer()
    tokenizer.fit_on_texts(a_sentences)

    return tokenizer

In [24]:
tokenizer = create_tokenizer(sentences)
vocab_size = len(tokenizer.word_index) + 1
print('размер словаря предложений ... %d' % vocab_size)

размер словаря предложений ... 26817
